In [83]:
import pandas as pd
path = 'C:/Users/ddamo/Desktop/3rd Evoleon/MOP-Code/env.txt'
envar = open(path,'r')
APIKEY = envar.read()

path = 'C:/Users/ddamo/Desktop/3rd Evoleon/MOP-Code/envG.txt'
envarG = open(path,'r')
APIKEYG = envarG.read()



In [4]:
import pandas as pd
import os
import requests
base_url= 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'

def fetch_data(base_url, dataset,api_key, num_records=99, offset= 0):
    all_records = [] 
    if dataset == popdata:
        filter = '&refine=age%3ATotal%20population'
    else:
        filter = ""

    max_offset = 300 # Maximum number of requests
    while True:
        # maximum limit check
        if offset > max_offset:
            break
# Create API request URL
        filters = f'{dataset}/records?limit={num_records}&offset={offset}{filter}'
        url = f' {base_url}{filters}&api_key={api_key}'
        
        # Start request
        try:
            result = requests.get(url, timeout=10)
            result. raise_for_status()
            records = result.json().get('results')
        except requests.exceptions.RequestException as e:
            raise Exception(f"API request failed: {e}")
        if records is None:
            break
        
        all_records.extend(records)
        if len(records) < num_records:
            break
            
        offset += num_records
        
        # next cycle offset offset + num_records
        # DataFrame all data
        
    df = pd.DataFrame(all_records)
    return df

In [85]:
popdata= 'city-of-melbourne-population-forecasts-by-small-area-2020-2040'
musicdata= 'live-music-venues'
df = fetch_data(base_url,popdata,APIKEY)
#print(df)
pop =pd.DataFrame()
df2 = fetch_data(base_url,musicdata,APIKEY)

pop["Area"]= df['geography']
pop['Year']= df['year']
pop['Population']= df['value']
pd.set_option('display.min_rows', None)
#print(pop)
#print(df2[1:2])
music = pd.DataFrame()
music['Name'] = df2['venue_name']
music['Address'] = df2['venue_address']
music['Type'] = df2['space_type']
music['Latitude'] = df2['lat']
music['Longitude']= df2['lon']

#music[['Longitude', 'Latitude']] = pd.DataFrame(music['Location'].tolist(), index=music.index)
#music.drop('Location', axis=1, inplace=True)


music


,Name,Address,Type,Latitude,Longitude
0,Fad Gallery,"Gnd & Flr1, 14 Corrs Lane, Melbourne, 3000",Bar,-37.811235,144.968335
1,Penny Blue,"2 Driver Lane, Melbourne, 3000",Bar,-37.812821,144.962989
2,Vodka Temple,"Ground, 160-162 Lygon Street, Carlton, 3053",Nightclub,-37.803452,144.966638
3,Union Electric,"13 Heffernan Lane, Melbourne, 3000",Bar,-37.811773,144.966689
4,Three Crowns Hotel,"361-365 Victoria Street, West Melbourne, 3003",Hotel,-37.805288,144.949490
5,Seamstress Restaurant and Bar,"113 Lonsdale Street, Melbourne, 3000",Bar,-37.810462,144.968957
6,European Bier Café,"120 Exhibition Street, Melbourne, 3000",bar,-37.812853,144.971104
7,Arbory Bar and Eatery,"2 Flinders Walk, Melbourne, 3000",Bar,-37.818910,144.966793
8,Touche Hombre,"237 Lonsdale Street, Melbourne, 3000",Bar,-37.811502,144.965440
9,AIM (Australian Institute of Music),"Gnd & Levels 1 -3, 120 King Street, Melbourne,...",Recording,-37.817262,144.956225


In [86]:
from geopy.geocoders import GoogleV3

def get_suburb_name(api_key, lat, lon):
    
    #Authenticate Your key and initialise Geocoding API
    geolocator = GoogleV3(api_key=api_key)

    #Catches errors
    try:
        location = geolocator.reverse((lat, lon), language='en')
        suburb = None
        #Searches address components for locaility, if its there it takes that and inputs in suburb
        for component in location.raw['address_components']:
            if 'locality' in component['types']:
                suburb = component['long_name']
                break
        #Returns suburb once its ran through it all
        return suburb
    #Catches errors
    except Exception as e:
        print(f"Error: {e}")
        return None
    
#Calling the function to get suburb names based on lat,lon
music['Suburb'] = music.apply(lambda row: get_suburb_name(APIKEYG, row['Latitude'], row['Longitude']), axis=1)



In [110]:
music

,Name,Address,Type,Latitude,Longitude,Suburb
0,Fad Gallery,"Gnd & Flr1, 14 Corrs Lane, Melbourne, 3000",Bar,-37.811235,144.968335,Melbourne
1,Penny Blue,"2 Driver Lane, Melbourne, 3000",Bar,-37.812821,144.962989,Melbourne
2,Vodka Temple,"Ground, 160-162 Lygon Street, Carlton, 3053",Nightclub,-37.803452,144.966638,Carlton
3,Union Electric,"13 Heffernan Lane, Melbourne, 3000",Bar,-37.811773,144.966689,Melbourne
4,Three Crowns Hotel,"361-365 Victoria Street, West Melbourne, 3003",Hotel,-37.805288,144.949490,West Melbourne
5,Seamstress Restaurant and Bar,"113 Lonsdale Street, Melbourne, 3000",Bar,-37.810462,144.968957,Melbourne
6,European Bier Café,"120 Exhibition Street, Melbourne, 3000",bar,-37.812853,144.971104,Melbourne
7,Arbory Bar and Eatery,"2 Flinders Walk, Melbourne, 3000",Bar,-37.818910,144.966793,Melbourne
8,Touche Hombre,"237 Lonsdale Street, Melbourne, 3000",Bar,-37.811502,144.965440,Melbourne
9,AIM (Australian Institute of Music),"Gnd & Levels 1 -3, 120 King Street, Melbourne,...",Recording,-37.817262,144.956225,Melbourne


In [124]:
MB = 0
NM = 0
SB = 0
SY = 0
CA = 0
DL = 0
EM = 0
WM = 0
PV = 0
SW = 0
KE = 0
NA = 0
#print(music["Suburb"])
for index, row in music.iterrows():
    if row["Suburb"] == "Melbourne":
        MB += 1
    elif row["Suburb"] == "North Melbourne":
        NM += 1
    elif row["Suburb"] == "Southbank":
        SB += 1
    elif row["Suburb"] == "South Yarra":
        SY += 1
    elif row["Suburb"] == "Carlton":
        CA += 1
    elif row["Suburb"] == "Docklands":
        DL += 1
    elif row["Suburb"] == "East Melbourne":
        EM += 1
    elif row["Suburb"] == "West Melbourne":
        WM += 1
    elif row["Suburb"] == "Parkville":
        PV += 1
    elif row["Suburb"] == "South Wharf":
        SW += 1
    elif row["Suburb"] == "Kensington":
        KE += 1
    else:
        NA += 1

In [125]:
print(MB)

print(NM)
print(SB)
print(SY)
print(CA)
print(DL)
print(EM)
print(WM)
print(PV)
print(SW)
print(KE)
print(NA)

Venue_total = MB + NM + SB + SY + CA + DL + EM + WM + PV + SW + KE + NA
print(Venue_total)

147
8
27
2
14
10
3
4
4
5
2
1
227


In [127]:
suburb_counts = {
    "Melbourne": 0,
    "North Melbourne": 0,
    "Southbank": 0,
    "South Yarra": 0,
    "Carlton": 0,
    "Docklands": 0,
    "East Melbourne": 0,
    "West Melbourne": 0,
    "Parkville": 0,
    "South Wharf": 0,
    "Kensington": 0,
    "Other": 0  
}

for index, row in music.iterrows():
    suburb = row["Suburb"]
    if suburb in suburb_counts:
        suburb_counts[suburb] += 1
    else:
        suburb_counts["Other"] += 1

In [135]:

popsorted = pop.sort_values(by=['Area', 'Year'])
print(popsorted)


                             Area  Year  Population
106                       Carlton  2021       17185
7                         Carlton  2022       17629
107                       Carlton  2023       17840
108                       Carlton  2024       18729
109                       Carlton  2025       20041
213                       Carlton  2026       20743
214                       Carlton  2027       21679
110                       Carlton  2028       22836
8                         Carlton  2029       24142
111                       Carlton  2030       25414
215                       Carlton  2031       26375
112                       Carlton  2032       27540
9                         Carlton  2033       28540
113                       Carlton  2034       30268
10                        Carlton  2035       32053
11                        Carlton  2036       33358
216                       Carlton  2037       34736
114                       Carlton  2038       34656
12          